In [6]:
import json
import pandas as pd
from pprint import pprint
from pathlib import Path

data = Path('data')

# Load the json file
with (data / 'franchises.json').open() as f:
	franchises = json.load(f)

# Convert the franchises list to a dataframe
df = pd.DataFrame(franchises)

# Show the dataframe
df = df.drop(columns=['animes', 'auto'])
df = df.sort_values(by=['my_score'], ascending=False)
df

[{'animes': [{'aired': {'from': '2007-09-01T00:00:00+00:00',
                        'prop': {'from': {'day': 1, 'month': 9, 'year': 2007},
                                 'to': {'day': None,
                                        'month': None,
                                        'year': None}},
                        'string': 'Sep 1, 2007',
                        'to': None},
              'airing': False,
              'approved': True,
              'background': '1.01: An additional 266 enhancements from 1.0. '
                            '1.11: 2 minutes longer than 1.0 and 1.01 and '
                            'created by processing the master digital data '
                            'instead of the 35mm film. In 2007, the film was '
                            'awarded the DCAJ Chairperson Award during the '
                            '22nd Digital Contents Grand Prix. In 2008, the '
                            'film won the Tokyo International Anime Fair award '
 

,title,episodes,score,my_score,genres,themes,demographics
2,"""Oshi no Ko""",11,8.830000,10.000000,"[Supernatural, Drama]","[Reincarnation, Showbiz]",[Seinen]
54,Tenkuu no Shiro Laputa,1,8.260000,10.000000,"[Fantasy, Adventure, Award Winning, Romance, S...",[],[]
22,Howl no Ugoku Shiro,1,8.660000,10.000000,"[Fantasy, Drama, Adventure, Award Winning, Rom...",[],[]
53,Tengoku Daimakyou,13,8.230000,10.000000,"[Mystery, Adventure, Sci-Fi]",[Survival],[Seinen]
36,Mononoke Hime,1,8.670000,10.000000,"[Fantasy, Award Winning, Adventure, Action]",[],[]
57,Vinland Saga,49,8.757959,9.979592,"[Adventure, Action, Drama]","[Gore, Historical]",[Seinen]
39,Nichijou:,27,8.420370,9.925926,[Comedy],"[Gag Humor, School]",[Shounen]
48,Shingeki no Kyojin:,89,8.678876,9.539326,"[Award Winning, Action, Drama, Suspense]","[Gore, Military, Survival]",[Shounen]
42,One Punch Man,24,8.000000,9.500000,"[Action, Comedy]","[Super Power, Parody, Adult Cast]",[Seinen]
18,Fullmetal Alchemist:,115,8.660957,9.113043,"[Fantasy, Drama, Adventure, Award Winning, Act...",[Military],[Shounen]
